In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.functional import normalize
from tqdm import tqdm

In [81]:
def read_data(file):
    return pd.read_csv(file, delimiter=',')

In [82]:
def split_data(data):
    x = data[:, :5]
    y = data[:, 5]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=37)
    return x_train, x_test, y_train, y_test

def transform(data):
    data = torch.tensor(data, dtype=torch.float32)
    # data = normalize(data)
    return data

In [83]:
def preprocess(filename):
    df_raw = read_data(filename)
    print(df_raw.head())
    d_raw = df_raw.to_numpy() # data to binary?
    x, x_test, y, y_test = split_data(d_raw)

    x, x_test = transform(x), transform(x_test)
    
    return x, x_test, y, y_test

In [124]:
datafile = 'data/basic_strategy_agent.csv'
x_train, x_test, y_train, y_test = preprocess(datafile)

   Player sum  Dealer card  Usable ace  Double down allowed  Split allowed  \
0           9            4           0                    1              0   
1          16           10           0                    1              0   
2          14            4           0                    1              0   
3          20            8           0                    1              1   
4          13            3           0                    1              0   

   Action  
0       2  
1       1  
2       0  
3       0  
4       0  


In [85]:
def one_hot_encode(y, n):
    return torch.eye(n)[y]

def combine_input_and_label(x, y, n):
    y_one_hot = one_hot_encode(y, n)
    return torch.concat((x, y_one_hot), 1)

def generate_neg_labels(y, n):
    y_ = y.copy()
    for i in range(len(y)):
        negative_labels = [j for j in range(n) if j != y[i]]
        y_[i] = np.random.choice(negative_labels)

    return y_

In [103]:

x_pos = combine_input_and_label(x_train, y_train, 4)

y_neg = generate_neg_labels(y_train, 4)
x_neg = combine_input_and_label(x_train, y_neg, 4)

y_train_OH = one_hot_encode(y_train, 4)
y_test_OH = one_hot_encode(y_test, 4)


print(f"x_pos[0]: {x_pos[0]}")
print(f"x_neg[0]: {x_neg[0]}")

x_pos[0]: tensor([15.,  4.,  0.,  1.,  0.,  1.,  0.,  0.,  0.])
x_neg[0]: tensor([15.,  4.,  0.,  1.,  0.,  0.,  1.,  0.,  0.])


## Backpropegation

In [94]:
def train(model, x, y, epochs, loss_function, optimizer, batch_size):
    
    model.train()
    num_batches = len(x) // batch_size
    
    for epoch in range(epochs):
        losses = []
        for batch in tqdm(range(num_batches)):
            start_idx = batch * batch_size
            end_idx = start_idx + batch_size
            
            x_batch = x[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            
            predictions = model(x_batch)
            loss = loss_function(predictions, y_batch)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print(f"epoch: {epoch + 1}/{epochs} avg. loss: {sum(losses)/len(losses)}")

In [137]:
def test(model, x_test, y_test, loss_function):
    
    model.eval()
    
    with torch.no_grad():
        predictions = model(x_test)
        test_loss = loss_function(predictions, y_test).item()
        
    print(f"avg. test loss: {test_loss}")   
    
def predict_action(model, x):
    model.eval()
    
    with torch.no_grad():
        predictions = model(x)
        return predictions.argmax(1)

In [147]:
from backpropNN import BackpropNN

input_size = 5
output_size = 4

# hyperparameters
hidden_size = 2000
activation_function = nn.ReLU()

# fully connected neural network with 4 hidden layers
backprop_model = BackpropNN(input_size, output_size, hidden_size, activation_function)

In [151]:
# hyperparameters
epochs = 2
batch_size = 32
learning_rate = 0.005

optimizer = torch.optim.Adam(backprop_model.parameters(), lr=learning_rate);
loss_fn = nn.CrossEntropyLoss()

train(backprop_model, x_train, y_train_OH, epochs, loss_fn, optimizer, batch_size)

100%|██████████| 379/379 [00:41<00:00,  9.08it/s]


epoch: 1/2 Avg. loss: 0.9681648470481027


100%|██████████| 379/379 [00:42<00:00,  9.00it/s]

epoch: 2/2 Avg. loss: 0.9680799989712899


In [152]:
test(backprop_model, x_test, y_test_OH, loss_fn)

avg. test loss: 0.9507748484611511


In [153]:
# print(torch.from_numpy(y_test))
# print(predict_action(backprop_model, x_test))
test_error = 1 - predict_action(backprop_model, x_test).eq(torch.from_numpy(y_test)).float().mean().item()
print(test_error)

tensor([0, 1, 1,  ..., 1, 0, 0])
tensor([0, 0, 0,  ..., 0, 0, 0])
0.5006591975688934
